In [69]:
# import libraries

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Make spec files

In [70]:
from src.scripts import make_specs

# where specs will be saved
SPEC_DIR = '/Users/maedbhking/Documents/hbn_data'

make_specs.run(out_dir=SPEC_DIR)

created pydraml specs, saved to /Users/maedbhking/Documents/hbn_data
created participant specs, saved to /Users/maedbhking/Documents/hbn_data
created feature specs, saved to /Users/maedbhking/Documents/hbn_data
created target specs, saved to /Users/maedbhking/Documents/hbn_data


## Make train/test splits

In [59]:
import os
from src.scripts import make_train_test_split

DATA_DIR = '/Users/maedbhking/Documents/hbn_data/'

# outpath for train/test data
outpath = os.path.join(DATA_DIR, 'participants_train_test.csv')

# path to dataframe you want to split
fpath = os.path.join(DATA_DIR, 'Clinical_Diagnosis_Demographics.csv')

# which columns of dataframe do you want to stratify
columns_to_stratify = ['Sex', 'PreInt_Demos_Fam,Child_Race_cat']

# which column indicates participant identifiers
columns_to_keep = ['Identifiers']

make_train_test_split.run(fpath=fpath, 
                         outpath=outpath,
                         columns_to_stratify=columns_to_stratify,
                         columns_to_keep=columns_to_keep,
                         test_size=.2,
                         random_state=42,
                         )

train/test splits saved to /Users/maedbhking/Documents/hbn_data/participants_train_test.csv


In [19]:
import pandas as pd

df = pd.read_csv(outpath)
df.head(5)

,Identifiers,split
0,NDARMZ366UY8,train
1,NDARJK842BCN,train
2,NDARDB224XUD,train
3,NDARMN043YKC,train
4,NDARLE222HJ5,train


## Make model

In [68]:
import glob

# list spec files
list(glob.glob(os.path.join(SPEC_DIR, '*spec.json*')))

['/Users/maedbhking/Documents/hbn_data/target1-spec.json',
 '/Users/maedbhking/Documents/hbn_data/pydraml1-spec.json',
 '/Users/maedbhking/Documents/hbn_data/target_DX_01_Cat_binarize-spec.json',
 '/Users/maedbhking/Documents/hbn_data/features-Teacher_Measures-domains-Peer_Affiliation_and_Social_Acceptance_-Dishion_Teacher-spec.json',
 '/Users/maedbhking/Documents/hbn_data/features1-spec.json',
 '/Users/maedbhking/Documents/hbn_data/participant1-spec.json',
 '/Users/maedbhking/Documents/hbn_data/pydraml2-spec.json']

In [67]:
from src.scripts import make_model

# where spec files are saved
SPEC_DIR = '/Users/maedbhking/Documents/hbn_data/' # directory where spec files are stored
DATA_DIR = '/Users/maedbhking/Documents/hbn_data/' # directory where data are stored 
OUT_DIR = '/Users/maedbhking/Documents/hbn_data/test123' # directory where model features + spec should be saved

# fullpath to feature/target/participant specs
feature_spec = os.path.join(SPEC_DIR, 'features1-spec.json')
target_spec = os.path.join(SPEC_DIR, 'target1-spec.json')
participant_spec = os.path.join(SPEC_DIR, 'participant1-spec.json')
pydraml_spec = os.path.join(SPEC_DIR, 'pydraml1-spec.json')

# make model spec and features
make_model.run(
    feature_spec,
    target_spec,
    participant_spec,
    pydraml_spec,
    data_dir=DATA_DIR,
    out_dir=OUT_DIR
    )

creating /Users/maedbhking/Documents/hbn_data/test123
created new file: features-766723.csv and model spec file: model_spec-766723.json in /Users/maedbhking/Documents/hbn_data/test123


## Run model(s)

In [56]:
from src.scripts import run_model

model_spec = '' # fullpath to model spec 
features = '' # fullpath to features (filename should correspond to `filename` in `model_spec`)
OUT_DIR = '' # directory where model results should be saved.
CACHE_DIR = None # where model cache will be saved. Default is '~/.cache/pydra-ml/cache-wf/'

# run model
run_model.run(
            model_spec=model_spec, 
            features=features,
            out_dir=OUT_DIR,
            cache_dir=CACHE_DIR
            )

## Model Summary

In [ ]:
from src.scripts import make_summary
    
results = '' # fullpath to results (.pkl)
model_spec = '' # fullpath to model_spec file (if None, code checks the model output directory for model spec)
out_dir = '' # directory where model summary will be saved
methods = ['feature'] # feature interpretability based on feature or permuation

# make model summary
make_summary.run(
    results,
    model_spec,
    out_dir,
    methods
    )